# Modal Analysis ROMY - OMA Analysis 

https://dewesoft.com/blog/what-is-modal-analysis

 ### What is modal analysis?

Modal analysis is an indispensable tool in understanding the structural dynamics of objects - how structures and objects vibrate and how resistant they are to applied forces. The modal analysis allows machines and structures to be tested, optimized, and validated. 

Modal analysis is widely accepted in a broad range of applications in the automotive, civil engineering, aerospace, power generation, and musical instruments industries, among countless others.

Natural resonance frequencies of the objects and damping parameters can be calculated, and mode shapes can be visualized on an animated geometry of the measured objects. 

The collection of modal parameters - natural frequencies, damping, and mode shapes - is referred to as a Modal Mod

Modal analysis is used for civil structures to handle concerns regarding the impact of seismic activities and heavy winds. Such types of analyses often use response-only testing like OMA, where the structural excitations are provided only by ambient vibrations and external forces. 

For example, modal analysis can show how a tall apartment building might reveal critically high deflection levels for some bending or torsional modes. After such modes have been detected it is often possible to model and implement some measures that counteract or attenuate such characteristics.

The same applies to bridges where flutter effects can occur at certain wind speeds, which can force the structure into oscillation. An example of such a fatal flutter effect is the Tacoma Narrows bridge between Tacoma, Washington, and the Kitsap Peninsula, which opened in 1940 and collapsed just four months later due to the effects of wind.

 ### ODS and OMA

Both ODS (operational deflection shapes) and OMA (operational modal analysis) do not use external input forces but are based purely on response DOF measurements. Therefore, the Modal Test procedure is the same for ODS and OMA, but the analysis and results are different. 

ODS provides basic information about amplitude and phase information of the DOFs on the measured operational DUT and enables geometry animation of the deflection shapes.

OMA estimates a Modal Model (like EMA) with natural frequencies, damping, and mode shapes of the measured operational DUT.   

But while EMA estimates Modal Models from FRF data obtained with the use of force input signal(s), OMA estimates Modal Models from operational vibration measurements, for example, calculated Auto- and Cross-power Spectral Density functions (PSD and CPSD). 

Because OMA does not use input force signals the applied forces are unknown. Therefore, modal masses cannot be estimated, and the estimated mode shapes will be unscaled.

OMA can be used to estimate a modal model in situations where it is difficult to do EMA. For example, when monitoring running DUTs for health issues, when the size or location of the DUT makes it impractical to excite with external force, or when the operational structural conditions of the DUT must be analyzed.

## Import Libraries

In [1]:
# Import modules
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import PyOMA as oma
import obspy
import mplcursors

from scipy import signal
from obspy.clients.filesystem.sds import Client

In [2]:
if os.uname().nodename == 'lighthouse':
    root_path = '/home/andbro/'
    data_path = '/home/andbro/kilauea-data/'
    archive_path = '/home/andbro/freenas/'
elif os.uname().nodename == 'kilauea':
    root_path = '/home/brotzer/'
    data_path = '/import/kilauea-data/'
    archive_path = '/import/freenas-ffb-01-data/'

In [3]:
def __get_data_all(config):

    from tqdm.notebook import tqdm

    net, sta, loc, cha = config['seed'].split(".")

    st0 = obspy.Stream()

    for sta in tqdm(config['stations']):

        try:
            st = config['client'].get_waveforms(network=net, station=sta, location=loc, channel=cha,
                                                starttime=config['tbeg'], endtime=config['tend'])

            if len(st) == 0:
                raise "no data in stream"

            inv = obspy.read_inventory(config['inventories']+f"XX.{sta}.xml")

            st = st.remove_response(inventory=inv, output="ACC", plot=False)

            st.merge()

            st0 += st

        except:
            print(f" -> loading failed for: {sta}")

    return st0

In [4]:
config = {}

## Kermadec M7.0 Event
# config['tbeg'] = obspy.UTCDateTime("2023-03-16 00:50")
# config['tend'] = obspy.UTCDateTime("2023-03-16 03:30")

## PNG M7.0 Event
# config['tbeg'] = obspy.UTCDateTime("2023-04-02 18:00")
# config['tend'] = obspy.UTCDateTime("2023-04-02 20:30")

## Local Event
# config['tbeg'] = obspy.UTCDateTime("2023-04-02 20:15")
# config['tend'] = obspy.UTCDateTime("2023-04-02 20:19")

## Local Noise
config['tbeg'] = obspy.UTCDateTime("2023-04-08 00:00")
config['tend'] = obspy.UTCDateTime("2023-04-09 00:00")

## tractor
# config['tbeg'] = obspy.UTCDateTime("2023-03-18 10:00")
# config['tend'] = obspy.UTCDateTime("2023-03-18 12:00")

## LF event
# config['tbeg'] = obspy.UTCDateTime("2023-03-30 18:00")
# config['tend'] = obspy.UTCDateTime("2023-03-30 19:00")


# config['tbeg'] = obspy.UTCDateTime("2023-03-10 12:00")
# config['tend'] = obspy.UTCDateTime("2023-03-10 15:00")

config['path_to_figs'] = root_path+"Documents/ROMY/ModalAnalysis/figures/"

config['client'] = Client(archive_path+"ModalAnalysis/data/SDS")
config['client'] = Client(archive_path+"ModalAnalysis/data/SDS")

config['seed'] = "XX.RY??..HH*"

config['stations'] = ['RY01','RY02','RY03','RY04','RY05','RY06','RY07','RY08','RY09','RY10']
# config['stations'] = ['RY01','RY05','RY07','RY09']
# config['stations'] = ['RY01']

config['inventories'] = archive_path+"ModalAnalysis/data/StationXML/"

config['fmin'] = 0.1
config['fmax'] = 20.0

config['str_tbeg'] = f"{config['tbeg'].date}T{str(config['tbeg'].time).split('.')[0]}"
config['str_tend'] = f"{config['tend'].date}T{str(config['tend'].time).split('.')[0]}"

In [5]:
!ls 

EMA_Analysis.ipynb	 modal_analysis_from_felix.py  RY_CheckData.ipynb
event_png_m7_data.mseed  noise_data_2.mseed	       RY_PlotDataAll.ipynb
lf_event_data.mseed	 noise_data.mseed	       short_local_event.txt
make_stationxml.ipynb	 oma_anaylsis.ipynb	       tractor_data.mseed


## Load Data

In [ ]:
st0 = __get_data_all(config)

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
# st0 = obspy.read("./noise_data.mseed")
# st0 = obspy.read("./noise_data_2.mseed")
# st0 = obspy.read("./tractor_data.mseed")
# st0 = obspy.read("./event_png_m7_data.mseed")
# st0 = obspy.read("./lf_event_data.mseed")

# st0.write("./event_png_m7_data.mseed", format="MSEED")

In [ ]:
# st0.plot(equal_scale=False);

## Pre-Process Data

In [ ]:
st = st0.copy();

st = st.sort();

## bandpass filter
st = st.detrend("simple");
st = st.taper(0.1);
st = st.filter("bandpass", freqmin=config['fmin'], freqmax=config['fmax'], corners=8, zerophase=True);

## resample data
st = st.resample(config['fmax']);

## remove corrupted data
for tr in st:
    if tr.stats.station in ["RY03","RY04"]:
        st = st.remove(tr);

## extract sampling rate
fs = st[0].stats.sampling_rate # [Hz] Sampling Frequency

In [ ]:
st = st.trim(st[0].stats.starttime+300, st[0].stats.endtime-300)

In [ ]:
st.plot(equal_scale=False);

## Prepare 2D-Array for Analysis

2D array required for methods (columns=sensors, rows=data)

In [ ]:
cha = "Z"

data, names = [], []

st = st.sort();

for tr in st:
    if cha in tr.stats.channel:
        data.append(tr.data)
        names.append(tr.stats.station)

times = st[0].times()/3600
data = np.array(data).transpose()

colors = ["tab:blue","tab:orange","tab:green","tab:red","tab:purple", "tab:brown", "tab:pink", "tab:gray", "tab:green"]


fig, ax = plt.subplots(len(names), 1, figsize=(15,10), sharex=True)

for i, name in enumerate(names):
    # ax[i].plot(times, data[:,i]*1e6, label=name, color=colors[i])
    ax[i].plot(times, data[:,i]*1e6, label=name+".HH"+cha)
    ax[i].legend(loc=1)
    ax[i].set_ylabel(r"acc. ($\mu$m/s$^2$)")

ax[len(names)-1].set_xlabel("Time (hours)")

plt.show();

# fig.savefig(config['path_to_figs']+f"ModAna_noise_{config['str_tbeg']}_{config['str_tend']}_traces.png", format="png", dpi=200, bbox_inches='tight');

In [ ]:
# Run FDD
FDD = oma.FDDsvp(data,  fs, df=0.01, pov=0.5)

ff = np.arange(0, FDD[1]['Data']['Samp. Freq.']/2+FDD[1]['Data']['Freq. Resol.'], FDD[1]['Data']['Freq. Resol.'])

sv = {}
for i, name in enumerate(names):
    sv[name] = FDD[1]['Singular Values'][i][i].real

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15,5))

font = 12

for key in sv.keys():
    ax.plot(ff,10*np.log10(sv[key]), label=key)

# ax.legend(loc=1, ncols=4)
ax.grid(which="both", alpha=0.5)
ax.set_xscale("log")

ax.set_xlim(config['fmin'], config['fmax']/2)
ax.set_ylim(bottom=-200)

ax.set_xlabel("Frequency (Hz)", fontsize=font)
ax.set_ylabel(r"dB (m$^2$/s$^2$/Hz)", fontsize=font)
ax.set_title(f"Singular valules plot - {cha}", fontsize=font+1)

plt.show();

# fig.savefig(config['path_to_figs']+f"ModAna_noise_{config['str_tbeg']}_{config['str_tend']}_svd.png", format="png", dpi=200, bbox_inches='tight');

In [ ]:
# ff = np.arange(0, FDD[1]['Data']['Samp. Freq.']/2+FDD[1]['Data']['Freq. Resol.'], FDD[1]['Data']['Freq. Resol.'])

# sv = {}
# dat0 = data[:,0].transpose()
# for i, name in enumerate(names):
#     dat1 = data[:,i:i+1].transpose()
#     dat = np.vstack((dat0, dat1))
#     FDD = oma.FDDsvp(dat.transpose(),  fs, df=0.01, pov=0.5)
#     sv[name] = FDD[1]['Singular Values'][1][1].real

# fig, ax = plt.subplots(1, 1, figsize=(15,5))

# font = 14

# for key in sv.keys():
#     ax.plot(ff,10*np.log10(sv[key]), label=key)

# ax.legend(loc=1, ncols=4)
# ax.grid(which="both", alpha=0.5)
# ax.set_xscale("log")

# ax.set_xlim(config['fmin'], config['fmax']/2)
# ax.set_ylim(bottom=-200)

# ax.set_xlabel("Frequency (Hz)")
# ax.set_ylabel(r"dB (m$^2$/s$^2$/Hz)")
# ax.set_title("Singular valules plot")

# plt.show();

# Frequency Domain Methods

In [ ]:
# Define list/array with the peaks identified from the plot
# FreQ = [0.89, 2.6, 4.1, 5.27, 6] # identified peaks
FreQ = [3.8] # identified peaks

# Extract the modal properties
Res_FDD = oma.FDDmodEX(FreQ, FDD[1])
Res_EFDD = oma.EFDDmodEX(FreQ, FDD[1], method='EFDD', npmax=35, plot=True)
Res_FSDD = oma.EFDDmodEX(FreQ, FDD[1], method='FSDD', npmax=15, MAClim=0.95, plot=True)

In [ ]:
print(f"RES_FDD: f = {Res_FDD['Frequencies']}")

print(f"RES_EFDD: f = {list(Res_EFDD['Frequencies'])}  d = {list(Res_EFDD['Damping'])}")

print(f"RES_FSDD: f = {list(Res_FSDD['Frequencies'])}  d = {list(Res_FSDD['Damping'])}")



# Time Domain Methods

stability diagrams to check if poles are stable and if they are similar

In [ ]:
br = 15

In [ ]:
# Run SSI covariance
SSIcov= oma.SSIcovStaDiag(data, fs, br)

# Extract the modal properties
Res_SSIcov = oma.SSIModEX(FreQ, SSIcov[1])

In [ ]:
# Run SII data
SSIdat = oma.SSIdatStaDiag(data, fs, br)
# SSIdat = oma.SSIdatStaDiag(data, fs, br, ordmax=60, lim=(0.01, 0.05, 0.02, 0.1))

# Extract the modal properties
Res_SSIdat= oma.SSIModEX(FreQ, SSIdat[1])

## Modal model validation

In [ ]:
MS_FDD = Res_FDD['Mode Shapes'].real
MS_EFDD = Res_EFDD['Mode Shapes'].real
MS_FSDD = Res_FSDD['Mode Shapes'].real
MS_SSIcov = Res_SSIcov['Mode Shapes'].real
MS_SSIdat = Res_SSIdat['Mode Shapes'].real

_nch = data.shape[1]

### Modal assurance criteria (MAC)

The Modal Assurance Criterion Analysis (MAC) analysis is used to determine the similarity of two-mode shapes.

The MAC number is defined as a scalar constant, between 0 and 1, expressing the degree of consistency between two mode shapes.

In practice, any value between 0.9 and 1.0 is considered a good correlation. Below 0.7 is considered to indicate a bad correlation.

In [ ]:
MAC = np.reshape(
        [oma.MaC(FI_ex[:,l], MS_FSDD[:,k]).real for k in range(_nch) for l in range(_nch)], # (_nch*_nch) list of MAC values
        (_nch, _nch)) # new (real) shape (_nch x _nch) of the MAC matrix


CrossMAC determines the consistency, or linearity, between estimated mode shape vectors from different estimations, different data sets

CrossMAC is a good tool to use for comparison between different experimentally determined mode shapes. For example, it might be interesting to compare two sets of measurements that are using different reference DOF locations and see how consistent the modes are between them. Or, to compare two different curve-fitting algorithms used on the same set of measurements. 

In addition to a comparison of different experimentally determined mode shapes, the CrossMAC can also be used to compare a set of experimentally determined mode shapes with a set of analytically determining mode shapes from Finite Element Analysis (FEA). The latter is used in Correlation Analysis. 

In [ ]:
crossMAC = np.reshape(
        [oma.MaC(MS_SSIcov[:,l], MS_SSIdat[:,k]).real for k in range(_nch) for l in range(_nch)], # (_nch*_nch) list of MAC values
        (_nch,_nch)) # new (real) shape (_nch x _nch) of the MAC matrix

In [ ]:
col = ["mode I", "mode II"]
# col = ["mode I", "mode II", "mode III", "mode IV", "mode V"]

MAC = pd.DataFrame(MAC, columns=col, index=col)
crossMAC = pd.DataFrame(crossMAC, columns=col, index=col)

## Plotting

In [ ]:
fig, ax = plt.subplots()
sns.heatmap(MAC,cmap="jet",ax=ax,annot=True, fmt='.3f',)
fig.tight_layout()
plt.show()

fig, ax1 = plt.subplots()
sns.heatmap(crossMAC,cmap="jet", ax=ax1, annot=True, fmt='.3f',)
fig.tight_layout()
plt.show()

In [ ]:
df = pd.DataFrame()
for tr in st:
    
    if tr.stats.station in ["RY05", "RY07", "RY09"]:
        df[tr.stats.station] = tr.data
    
df.to_csv('./short_local_event.txt', sep="\t", header=0, index=None, mode="w")
df

In [ ]:
# from obspy.clients.fdsn import Client

# tbeg, tend = st[0].stats.starttime, st[0].stats.endtime

# c= Client("BGR")

# fur = c.get_waveforms("GR", "FUR", "", "HHZ", tbeg, tend, attach_response=True)

# # ## bandpass filter
# fur = fur.detrend("simple");
# fur = fur.taper(0.1);
# fur = fur.filter("bandpass", freqmin=config['fmin'], freqmax=config['fmax'], corners=8, zerophase=True);

# # ## resample data
# fur = fur.resample(config['fmax']);

# fur.plot();

In [ ]:
from obspy.clients.fdsn import Client

tbeg, tend = st[0].stats.starttime, st[0].stats.endtime

c= Client("LMU")

ffb1 = c.get_waveforms("BW", "FFB1", "", "HHZ", tbeg, tend, attach_response=True)

ffb1.remove_response("ACC")

# ## bandpass filter
ffb1 = ffb1.detrend("simple");
ffb1 = ffb1.taper(0.1);
ffb1 = ffb1.filter("bandpass", freqmin=config['fmin'], freqmax=config['fmax'], corners=8, zerophase=True);

# ## resample data
# ffb1 = ffb1.resample(config['fmax']);

ffb1.plot();

ffb1.write("./ffb1_m7.mseed")

In [ ]:
from scipy.signal import welch, hann

tr = ffb1[0]

twin = 100

nblock =  int(tr.stats.sampling_rate*twin)
overlap = int(0.5*nblock)
win = hann(nblock, True)

ffx, Pxx = welch(tr.data, fs=tr.stats.sampling_rate, 
                window=win, noverlap=overlap, nfft=nblock,
                scaling="density",
                return_onesided=True)

In [ ]:
plt.semilogx(ffx, 10*np.log10(Pxx))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15,5))

font = 12

for key in sv.keys():
    ax.plot(ff,10*np.log10(sv[key]), label=key)

ax.plot(ffx, 10*np.log10(Pxx)-130, 'k')
    
# ax.legend(loc=1, ncols=4)
ax.grid(which="both", alpha=0.5)
ax.set_xscale("log")

ax.set_xlim(config['fmin'], config['fmax']/2)
ax.set_ylim(bottom=-200)

ax.set_xlabel("Frequency (Hz)", fontsize=font)
ax.set_ylabel(r"dB (m$^2$/s$^2$/Hz)", fontsize=font)
ax.set_title(f"Singular valules plot - {cha}", fontsize=font+1)

plt.show();

# fig.savefig(config['path_to_figs']+f"ModAna_noise_{config['str_tbeg']}_{config['str_tend']}_svd.png", format="png", dpi=200, bbox_inches='tight');